# upgrade nerual network from model5 
Of course. Your current script is well-structured and uses strong techniques like BatchNormalization and LeakyReLU. To get better results, we can focus on three key areas:

Switching to a Multi-Output Model: Instead of training a separate model for each target variable, we can train a single, more efficient model that predicts all of them simultaneously. This allows the model to learn shared patterns in the data that can improve predictions across all outputs.

Feature Engineering: We can create more meaningful input features from the existing data. For example, calculating the age of a dam is likely more predictive than just its completion year.

Refining the Architecture: Your network is quite deep. We can adjust its shape to a more traditional "funnel" structure, which can sometimes improve the flow of information.

Here is the upgraded code incorporating these changes.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer # Import imputer
from sklearn.pipeline import Pipeline # Import pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import openpyxl
import os
from datetime import datetime

# --- Configuration ---
DATA_FILE = '../regression_data.csv' # Make sure this path is correct

# Original inputs
INPUT_COLUMNS = [
    'state', 'downstream_hazard_potential', 'owner_type', 'dam_type',
    'primary_purpose_s', 'eap', 'latitude', 'longitude'
]

# Engineered features will be added to this list
FEATURE_ENGINEERED_COLUMNS = [
    'dam_age', 'years_since_modified'
]

# All target variables for the multi-output model
TARGET_COLUMNS = [
    'dam_height', 'max_storage_ac_ft', 'surface_area_acres',
    'incident_date_year', 'incident_date_month', 'incident_date_day',
    'incident_time_hour', 'number_of_people_evacuated',
    'number_of_habitable_structures_evacuated',
    'number_of_habitable_structures_flooded',
    'volume_released_at_failure_ac_ft', 'incident_duration'
]

# --- Training Configuration ---
EPOCHS = 200 # Increased epochs for potentially more complex task
EARLY_STOPPING_PATIENCE = 20
REDUCE_LR_PATIENCE = 8

# --- UPGRADED: Multi-Output Model Building Function ---
def build_multi_output_model(input_shape, num_outputs):
    """Builds an upgraded, multi-output regression model with a funnel architecture."""
    # --- Define Hyperparameters ---
    units_1 = 512
    units_2 = 256
    units_3 = 128
    units_4 = 64 # Added a smaller fourth layer
    l2_reg = 0.001
    dropout_rate = 0.3
    learning_rate = 0.001

    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=input_shape),

        # Block 1
        tf.keras.layers.Dense(units=units_1, kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate),

        # Block 2
        tf.keras.layers.Dense(units=units_2, kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate),
        
        # Block 3
        tf.keras.layers.Dense(units=units_3, kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate),
        
        # Block 4 (New smaller layer)
        tf.keras.layers.Dense(units=units_4, kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
        tf.keras.layers.LeakyReLU(alpha=0.1),
        tf.keras.layers.BatchNormalization(),
        # No dropout on the layer before the output

        # Output Layer for Multi-Output Regression
        tf.keras.layers.Dense(num_outputs, activation='linear')
    ])
    
    # Use AdamW optimizer which can be more robust with weight decay
    optimizer = tf.keras.optimizers.AdamW(learning_rate=learning_rate, weight_decay=0.01)

    model.compile(
        optimizer=optimizer,
        loss='mean_squared_error',
        metrics=['mae']
    )
    return model

# --- Main Execution ---
if __name__ == "__main__":
    try:
        df = pd.read_csv(DATA_FILE)
    except FileNotFoundError:
        print(f"Error: The data file '{DATA_FILE}' was not found.")
        exit()

    # --- 1. Feature Engineering ---
    print("Engineering new features...")
    current_year = datetime.now().year
    # Ensure year columns are numeric, coercing errors to NaT (which becomes NaN)
    df['year_completed'] = pd.to_numeric(df['year_completed'], errors='coerce')
    df['year_modified'] = pd.to_numeric(df['year_modified'], errors='coerce')
    
    df['dam_age'] = current_year - df['year_completed']
    # If year_modified is missing, assume it's the same as completion year
    df['years_since_modified'] = current_year - df['year_modified'].fillna(df['year_completed'])
    print("✅ Feature engineering complete.")
    
    # Define final input features
    all_input_features = INPUT_COLUMNS + FEATURE_ENGINEERED_COLUMNS
    
    # Drop rows where any of the target columns are missing
    df_clean = df.dropna(subset=TARGET_COLUMNS)
    
    if len(df_clean) < 50:
        print("Error: Not enough data to train after dropping rows with missing targets. Exiting.")
        exit()
        
    X = df_clean[all_input_features]
    y = df_clean[TARGET_COLUMNS]

    # --- 2. Preprocessing Pipeline (with Imputation) ---
    categorical_features = X.select_dtypes(include=['object', 'category']).columns
    numerical_features = X.select_dtypes(include=np.number).columns
    
    # Create pipelines for numeric and categorical features to handle missing values
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')), # Use median for robustness to outliers
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough'
    )

    # --- 3. Train-Test Split ---
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    # --- 4. Model Training ---
    input_shape = (X_train_processed.shape[1],)
    num_outputs = y_train.shape[1]
    
    model = build_multi_output_model(input_shape, num_outputs)
    model.summary() # Print model architecture

    print("\n--- Training the multi-output model... ---")
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=EARLY_STOPPING_PATIENCE, restore_best_weights=True)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=REDUCE_LR_PATIENCE, min_lr=1e-6)

    history = model.fit(
        X_train_processed,
        y_train,
        epochs=EPOCHS,
        validation_split=0.2,
        callbacks=[early_stopping, reduce_lr],
        verbose=1 # Show training progress
    )
    print("✅ Training complete.")

    # --- 5. Evaluation & Reporting ---
    y_pred = model.predict(X_test_processed)
    # y_pred is now a 2D array, so convert it to a DataFrame for easier handling
    y_pred_df = pd.DataFrame(y_pred, columns=TARGET_COLUMNS, index=y_test.index)
    
    model_metrics_data = []

    print("\n--- Generating reports and plots for each target ---")
    for i, target_name in enumerate(TARGET_COLUMNS):
        actuals = y_test[target_name]
        predictions = y_pred_df[target_name]
        
        mae = mean_absolute_error(actuals, predictions)
        mse = mean_squared_error(actuals, predictions)
        r2 = r2_score(actuals, predictions)
        
        metrics_result = {
            'Model Output': target_name,
            'Task Type': 'Regression',
            'MAE': mae,
            'MSE': mse,
            'R2_Score': r2
        }
        model_metrics_data.append(metrics_result)

        # Generate Actual vs. Predicted Plot
        plt.figure(figsize=(8, 8))
        plt.scatter(actuals, predictions, alpha=0.5)
        plt.plot([actuals.min(), actuals.max()], [actuals.min(), actuals.max()], '--r', linewidth=2)
        plt.title(f'Actual vs. Predicted for {target_name}')
        plt.xlabel('Actual Values')
        plt.ylabel('Predicted Values')
        plot_filename = f'plot_{target_name}.svg'
        plt.tight_layout()
        plt.savefig(plot_filename, format='svg')
        plt.close()

    # Save summary metrics
    metrics_df = pd.DataFrame(model_metrics_data)
    metrics_filename = 'multi_output_model_performance.xlsx'
    metrics_df.to_excel(metrics_filename, index=False)
    print(f"\n✅ All performance metrics saved to '{metrics_filename}'.")
    
    # Save detailed report with all predictions
    detailed_results = X_test.copy()
    for col in TARGET_COLUMNS:
        detailed_results[f'actual_{col}'] = y_test[col]
        detailed_results[f'predicted_{col}'] = y_pred_df[col]
        
    detailed_results.to_excel('multi_output_detailed_report.xlsx', index=False)
    print(f"✅ Detailed report saved to 'multi_output_detailed_report.xlsx'.")

    print("\nAll tasks complete.")


Engineering new features...
✅ Feature engineering complete.

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               5632      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 batch_normalization (Batch  (None, 512)               2048      
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0    